# PSTH Analysis Pipeline
Welcome to the script for  the PSTH analysis. Please follow the upcoming steps in this notebook for further instructions. 

## 1) Import the packages required for running the script
Please run the block of code to import the Python packages that are required for running the rest of this script. 

In [1]:
#Import required packages
import os
import ipywidgets
from ipywidgets import Layout, HBox, VBox
from IPython.display import display
from PSTH_analysis import psth_analysis

## 2) Enter parameters for the PSTH Analysis

In [2]:
#Main path for the data
mainPath_html = ipywidgets.HTML(
    value = "<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with '/' at the end) that is hierarchically right above the folders of the recording sessions</p>"
)

mainPath_input = ipywidgets.Text(
    value = "", 
    placeholder = "Enter path for data", 
    disabled = False
)
display(VBox([mainPath_html, mainPath_input]))

#Group to analyze
group_html = ipywidgets.RadioButtons(
    options=['Group 0', 'Group 1'],
    value='Group 0',
    disabled=False
)
display(VBox([
    ipywidgets.HTML(value = "<b> Select a group to be analyzed: </b>"), 
    group_html
]))

#Analysis method
analysis_html = ipywidgets.RadioButtons(
    options=['Electrode-wise', 'Unit-wise', 'Both'],
    value='Electrode-wise',
    disabled=False
)
display(VBox([
    ipywidgets.HTML(value = "<b> Analysis Method: </b>"), 
    analysis_html
]))


#User specific window
user_window_check_html = ipywidgets.Checkbox(
    value=False,
    description="Window PSTH Analysis",
    disabled=False
)
user_window_html = ipywidgets.IntRangeSlider(
    value=[0, 180],
    min=0,
    max=180,
    step=1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
display(VBox([
    ipywidgets.HTML(value = "<b> Specify a valid interval if you want to analyze a window as a whole: </b>"),
    ipywidgets.HTML(value = "Check the below box if you want to do specific window PSTH Analysis:"),
    user_window_check_html,
    ipywidgets.HTML(value = "Select your interval by sliding the knobs:"),
    user_window_html
]))

#Spike decision
decision_html = ipywidgets.RadioButtons(
    options=['All', 'Only good'],
    value='All',
    disabled=False
)
display(VBox([
    ipywidgets.HTML(value = "<b> Select whether analyze all spikes or those which are clustered under 'Good' in 'klustaviewa': </b>"), 
    decision_html
]))

#Window duration
window_duration_html = ipywidgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(VBox([
    ipywidgets.HTML(value = "<b> Window duration (min): </b>"), 
    ipywidgets.HTML(value = "Select duration of PSTH Analysis windows by sliding the knob."),
    window_duration_html
]))

#Analysis interval before FUS
pre_interval_html = ipywidgets.IntText(value = 20, disabled = False)
display(VBox([
    ipywidgets.HTML(value = "<b> Pre interval (ms): </b>"), 
    ipywidgets.HTML(value = "Enter the duration of pre interval before a stimulus"), 
    pre_interval_html
]))


#Analysis interval after FUS
post_interval_html = ipywidgets.IntText(value = 80, disabled = False)
display(VBox([
    ipywidgets.HTML(value = "<b> Post interval (ms): </b>"), 
    ipywidgets.HTML(value = "Enter the duration of post interval after a stimulus"), 
    post_interval_html
]))

#Bin size 
bin_size_html = ipywidgets.FloatText(value = 1, disabled = False)
display(VBox([
    ipywidgets.HTML(value = "<b> Bin size (ms): </b>"), 
    ipywidgets.HTML(value = "Enter size of the histogram bins that determines the width of the bars"), 
    bin_size_html
]))

#FUS On time
t_fus_on_html = ipywidgets.FloatText(value = 20, disabled = False)
display(VBox([
    ipywidgets.HTML(value = "<b> Timing of the FUS ON (min): </b>"), 
    ipywidgets.HTML(value = "Enter the time when FUS is activated"), 
    t_fus_on_html
]))


VBox(children=(HTML(value="<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with '/' at the end) that is hierarchically right above the folders of the recording sessions</p>"), Text(value='', placeholder='Enter path for data')))

VBox(children=(HTML(value='<b> Select a group to be analyzed: </b>'), RadioButtons(options=('Group 0', 'Group 1'), value='Group 0')))

VBox(children=(HTML(value='<b> Analysis Method: </b>'), RadioButtons(options=('Electrode-wise', 'Unit-wise', 'Both'), value='Electrode-wise')))

VBox(children=(HTML(value='<b> Specify a valid interval if you want to analyze a window as a whole: </b>'), HTML(value='Check the below box if you want to do specific window PSTH Analysis:'), Checkbox(value=False, description='Window PSTH Analysis'), HTML(value='Select your interval by sliding the knobs:'), IntRangeSlider(value=(0, 180), max=180)))

VBox(children=(HTML(value="<b> Select whether analyze all spikes or those which are clustered under 'Good' in 'klustaviewa': </b>"), RadioButtons(options=('All', 'Only good'), value='All')))

VBox(children=(HTML(value='<b> Window duration (min): </b>'), HTML(value='Select duration of PSTH Analysis windows by sliding the knob.'), IntSlider(value=10, max=50, min=1)))

VBox(children=(HTML(value='<b> Pre interval (ms): </b>'), HTML(value='Enter the duration of pre interval before a stimulus'), IntText(value=20)))

VBox(children=(HTML(value='<b> Post interval (ms): </b>'), HTML(value='Enter the duration of post interval after a stimulus'), IntText(value=80)))

VBox(children=(HTML(value='<b> Bin size (ms): </b>'), HTML(value='Enter size of the histogram bins that determines the width of the bars'), FloatText(value=1.0)))

VBox(children=(HTML(value='<b> Timing of the FUS ON (min): </b>'), HTML(value='Enter the time when FUS is activated'), FloatText(value=20.0)))

## 3) Initiate the PSTH Analysis
Please run the block of the code in order to excute the PSTH algorithm based on the input that you have provided above.

In [4]:
psthParameters = {} #Create a dictionary for storing PSTH parameters to pass to function
psthParameters['mainPath'] = str(mainPath_input.value)
print("Path to the data: {0}".format(psthParameters['mainPath']))
psthParameters['experiment_id'] = psthParameters['mainPath'].rpartition('_')[2].replace('/','')
print("Experiment id: {0}".format(psthParameters['experiment_id']))
if(str(group_html.value) == 'Group 0'):
    psthParameters['group'] = 0
else:
    psthParameters['group'] = 1

if(str(analysis_html.value) == 'Electrode-wise'):
    psthParameters['psth_all_electrodes'] = True
    psthParameters['psth_all_units'] = False
elif(str(analysis_html.value) == 'Unit-wise'):
    psthParameters['psth_all_electrodes'] = False
    psthParameters['psth_all_units'] = True
else:
    psthParameters['psth_all_electrodes'] = True
    psthParameters['psth_all_units'] = True

if(bool(user_window_check_html.value) == True):
    psthParameters['user_window'] = [True, int(user_window_html.value[0]), int(user_window_html.value[1])]
else:
    psthParameters['user_window'] = [False, 0, 0]

if(str(decision_html.value) == 'All'):
    psthParameters['decision'] = 'all'
else:
    psthParameters['decision'] = 'only_good'

psthParameters['window_duration'] = int(window_duration_html.value)
psthParameters['pre_interval_ms'] = float(pre_interval_html.value)
psthParameters['post_interval_ms'] = float(post_interval_html.value)
psthParameters['bin_size_ms'] = float(bin_size_html.value)
psthParameters['t_fus_on'] = float(t_fus_on_html.value)
print('FUS activation: {0}. minute\n'.format(psthParameters['t_fus_on']))

psth_analysis(psthParameters)

Path to the data: /media/yaniklab/05d01d78-2bd6-4a4e-b573-df49ccacb71c/2018_04_12_FUSs1_EphysM1_E-FUS_NBBB67/
Experiment id: NBBB67
FUS activation: 25.0. minute

Reading Stimulation Timestamps: /media/yaniklab/05d01d78-2bd6-4a4e-b573-df49ccacb71c/2018_04_12_FUSs1_EphysM1_E-FUS_NBBB67/FUS_Muscimol_180412_164651

Reading Stimulation Timestamps: /media/yaniklab/05d01d78-2bd6-4a4e-b573-df49ccacb71c/2018_04_12_FUSs1_EphysM1_E-FUS_NBBB67/FUS_Muscimol_180412_174703

Analyzing: /media/yaniklab/05d01d78-2bd6-4a4e-b573-df49ccacb71c/2018_04_12_FUSs1_EphysM1_E-FUS_NBBB67/


Electrode-wise PSTH Analysis
PSTH Analysis for electrode-0 is done
PSTH Analysis for electrode-1 is done
PSTH Analysis for electrode-2 is done
PSTH Analysis for electrode-3 is done
PSTH Analysis for electrode-4 is done
PSTH Analysis for electrode-5 is done
PSTH Analysis for electrode-6 is done
PSTH Analysis for electrode-7 is done
PSTH Analysis for electrode-8 is done
PSTH Analysis for electrode-9 is done
PSTH Analysis for elec

# Done!
Please wait until "PSTH Analysis is completed succesfully." is printed at the end of the above cell.

Notebook written by Abdulkadir Gokce in 07/2018. Please contact him or Mehmet Ozdas in case of any questions.